In [185]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import json
from datetime import datetime 
from alpaca_trade_api.rest import REST, TimeFrame
import tweepy
%matplotlib inline
from pathlib import Path
import csv
import re
import emoji
import nltk
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
from collections import Counter
from nltk import ngrams
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib as mpl
import spacy
from spacy import displacy
import warnings
warnings.filterwarnings('ignore')
nltk.download('vader_lexicon')
plt.style.use('seaborn-whitegrid')
mpl.rcParams['figure.figsize'] = [20.0, 10.0]
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sameerlakhe/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [131]:
# Load .env enviroment variables
load_dotenv()
bearer_token =  os.getenv('TWITTER_BEARER_TOKEN')
api_key =  os.getenv('TWITTER_API_KEY')
api_secret_key =  os.getenv('TWITTER_SECRET_KEY')
access_token =  os.getenv('TWITTER_ACCESS_TOKEN')
access_secret_token =  os.getenv('TWITTER_SECRET_TOKEN')


def get_api_keys():
    api_key =  os.getenv('TWITTER_API_KEY')
    api_secret_key =  os.getenv('TWITTER_SECRET_KEY')
    bearer_token =  os.getenv('TWITTER_BEARER_TOKEN')
    access_token =  os.getenv('TWITTER_ACCESS_TOKEN')
    access_secret_token =  os.getenv('TWITTER_SECRET_TOKEN')
    return api_key,api_secret_key,bearer_token,access_token,access_secret_token

def get_api_keys1():
    api_key =  os.getenv('access_token')
    api_secret_key =  os.getenv('access_token_secret')
    bearer_token =  os.getenv('TWITTER_BEARER_TOKEN')
    access_token =  os.getenv('consumer_key')
    access_secret_token =  os.getenv('consumer_secret')
    return api_key,api_secret_key,bearer_token,access_token,access_secret_token

api_key,api_secret_key,bearer_token,access_token,access_secret_token = get_api_keys()


In [130]:
url = "https://api.twitter.com/2/tweets/1535608320211427329?tweet.fields=created_at,attachments&expansions=author_id"

payload={}
headers = {
  'id': '1228393702244134912',
  'Authorization': 'Bearer '+ bearer_token,
  'Cookie': 'guest_id=v1%3A165495549908649152'
}

response = requests.request("GET", url, headers=headers, data=payload)
json_response = response.text
print(json_response)

{"data":{"text":"On this day in 1980: Steve Ballmer became Microsoft employee #24.\n\nHis shares are now worth $81 billion. https://t.co/tHc8wtldIG","author_id":"164425558","attachments":{"media_keys":["3_1535608317837443073"]},"created_at":"2022-06-11T13:02:08.000Z","id":"1535608320211427329"},"includes":{"users":[{"id":"164425558","name":"Jon Erlichman","username":"JonErlichman"}]}}


In [134]:
query = "BAYC"
client = tweepy.Client(bearer_token=bearer_token)
counts = client.get_recent_tweets_count(query=query,granularity="day")
for count in counts.data:
    print(count)

{'end': '2022-06-07T00:00:00.000Z', 'start': '2022-06-06T23:36:49.000Z', 'tweet_count': 251}
{'end': '2022-06-08T00:00:00.000Z', 'start': '2022-06-07T00:00:00.000Z', 'tweet_count': 48509}
{'end': '2022-06-09T00:00:00.000Z', 'start': '2022-06-08T00:00:00.000Z', 'tweet_count': 21305}
{'end': '2022-06-10T00:00:00.000Z', 'start': '2022-06-09T00:00:00.000Z', 'tweet_count': 20500}
{'end': '2022-06-11T00:00:00.000Z', 'start': '2022-06-10T00:00:00.000Z', 'tweet_count': 46016}
{'end': '2022-06-12T00:00:00.000Z', 'start': '2022-06-11T00:00:00.000Z', 'tweet_count': 24803}
{'end': '2022-06-13T00:00:00.000Z', 'start': '2022-06-12T00:00:00.000Z', 'tweet_count': 40333}
{'end': '2022-06-13T23:36:49.000Z', 'start': '2022-06-13T00:00:00.000Z', 'tweet_count': 28305}


In [136]:
def create_tweet(text):
    client = tweepy.Client(consumer_key=api_key,
        consumer_secret=api_secret_key,
        access_token=access_token,
        access_token_secret=access_secret_token)
    response=client.create_tweet(text=text)

#create_tweet("Hello World!")

In [39]:
def determine_file_name(query):
    file_name = "Data/"+query+".csv"
    return file_name

In [40]:
def search_tweets(query):
    client = tweepy.Client(bearer_token=bearer_token)
    response = client.search_recent_tweets(query=query)
    return response

query = "NFT"

response = search_tweets(query)
#for tweet in response.data:
   # print(tweet.text)

In [114]:
#this function writes the event data to a csv file to avoid calling the API multiple times
def write_csv(data_list, file_name):
    with open(file_name, mode='w', encoding='utf-8', newline='\n') as f:
        header = "Tweets"
        f.write("%s\n" % header)
        for data in data_list:
            f.write("%s\n" % data)

In [173]:
def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)

nltk.download('words')
words = set(nltk.corpus.words.words())


def remove_non_english_words(text):
    return " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())

def sanitize_tweet(tweet):
    sanitzed_tweet = tweet.replace("\n", " ")
    sanitzed_tweet = re.sub(r'@[A-za-z0-9]+','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r'#','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r'RT[\s]','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r'https?:\/\/\S+','',sanitzed_tweet)
    sanitzed_tweet = re.sub(r',+','',sanitzed_tweet)
    #sanitzed_tweet = re.sub(r'✅+','',sanitzed_tweet)
    #sanitzed_tweet = re.sub(r'🚨+','',sanitzed_tweet)
    #sanitzed_tweet = remove_emoji(sanitzed_tweet)
    #sanitzed_tweet = remove_non_english_words(sanitzed_tweet)
    sanitzed_tweet =  re.sub('[^A-Za-z0-9]+', ' ', sanitzed_tweet)
    return sanitzed_tweet

def remove_duplicates(tweet_list):
    unique_lines = set()
    for tweet in tweet_list:
        unique_lines.add(tweet)
    return list(unique_lines)  

[nltk_data] Downloading package words to
[nltk_data]     /Users/sameerlakhe/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [181]:
#get the max data
limit = 50

max_results = 100
tweet_list=[]
def search_tweets1(query,reload=False):
    client = tweepy.Client(bearer_token=bearer_token)
    file_name = determine_file_name(query)
    path = Path(file_name)
    
    i=1
    if (reload == False and path.is_file()):
        print(f'The file {file_name} exists')
        nft_df = pd.read_csv(file_name)
        return nft_df
    else:
        for tweet in tweepy.Paginator(client.search_recent_tweets,query=query,max_results=max_results).flatten(limit=limit):
            #print(str(i) +")" + tweet.text)
            i=i+1
            sanitized_tweet = sanitize_tweet(tweet.text)
            tweet_list.append(sanitized_tweet)
        #once the list is generated, write it to file
        unique_tweet_list = remove_duplicates(tweet_list)
        write_csv(unique_tweet_list,file_name)
    #return list as a dataframe
    return pd.DataFrame(unique_tweet_list)

query = "BAYC"
is_reload_needed = False
bayc_tweet_df = search_tweets1(query,is_reload_needed)

pd.set_option('max_colwidth', 800)
bayc_tweet_df


The file Data/BAYC.csv exists


,Tweets
0,bayc better than goblins
1,Here s a cute Honorary Monkey for Let s see if he likes it BAYC CheerfulMonkeys
2,How to Mutate a Classic Ape ETC BAYC to MAYC via
3,Bored Ape 7537 sells for 1024 ETH 1213265 WTF BAYC
4,BAYC
5,After observing the continual bottleneck issue with absurd gas fees on L1 especially for large NFT projects such as BAY
6,bayc rules
7,HOLY FUCK I finally got my crown ape speechless shaking LFGGGGG BAYC LFGG
8,MAYC BAYC FBM RT Trait swap modify your APE Dynamic NFT 2 0 Fancy Bears app is now and running
9,Grandpa Ape Country Club 3685 Rank 1624 Rare Trait Chef Hat 1 26 GACC


In [179]:
#query = "NFT"
#is_reload_needed = False
#tweet_df = search_tweets1(query,is_reload_needed)
#tweet_df

In [182]:
#dictionary to store the sentiment scores of the article
tweet_sentiment = {
        "tweet_compound": [],
        "tweet_pos": [],
        "tweet_neu": [],
        "tweet_neg": [],
        "tweet_sent": [],
        }

analyzer = SentimentIntensityAnalyzer()
# Get sentiment for the tweet
def get_sentiment_score(tweet_df):
    
    for index, row in tweet_df.iterrows():
        try:
            # Sentiment scoring with VADER
            text_sentiment = analyzer.polarity_scores(row["Tweets"])
            tweet_sentiment["tweet_compound"].append(text_sentiment["compound"])
            tweet_sentiment["tweet_pos"].append(text_sentiment["pos"])
            tweet_sentiment["tweet_neu"].append(text_sentiment["neu"])
            tweet_sentiment["tweet_neg"].append(text_sentiment["neg"])
            tweet_sentiment["tweet_sent"].append(text_sentiment["compound"])
        except AttributeError:
            pass
        
    return tweet_sentiment

    
# Attaching sentiment columns to the News DataFrame
tweet_sentiment = get_sentiment_score(bayc_tweet_df)
tweet_sentiment_df = pd.DataFrame(tweet_sentiment)
bayc_tweet_sentiment_df = bayc_tweet_df.join(tweet_sentiment_df)
bayc_tweet_sentiment_df.head()

,Tweets,tweet_compound,tweet_pos,tweet_neu,tweet_neg,tweet_sent
0,bayc better than goblins,0.4404,0.492,0.508,0.000,0.4404
1,Here s a cute Honorary Monkey for Let s see if he likes it BAYC CheerfulMonkeys,0.8020,0.451,0.549,0.000,0.8020
2,How to Mutate a Classic Ape ETC BAYC to MAYC via,0.0000,0.000,1.000,0.000,0.0000
3,Bored Ape 7537 sells for 1024 ETH 1213265 WTF BAYC,-0.7672,0.000,0.547,0.453,-0.7672
4,BAYC,0.0000,0.000,1.000,0.000,0.0000


In [ ]:
nltk.download('omw-1.4')
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    # Create a tokenized list of the words
    words = word_tokenize(re_clean)
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
    # Convert the words to lowercase
    # Remove the stop words
    tokens = [word.lower() for word in lem if word.lower() not in sw]
    return tokens

# Create a new tokens column for Bitcoin
def generate_tokens(df):
    list_tokens=[]
    for row in df["Tweets"]:
        tokens = tokenizer(row)
        list_tokens.append(tokens)
    return list_tokens

bayc_tokens = generate_tokens(bayc_tweet_df)
bayc_tweet_df['tokens'] = bayc_tokens
bayc_tweet_df.head()

#concat data from all the rows for tokens column
def get_big_token_string(df):
    big_str=''
    for row in df["tokens"]:
        list_string = ' '.join(row)
        big_str = big_str + list_string
    return big_str

# Generate the Bitcoin N-grams where N=2
big_string_bayc_tokens = get_big_token_string(bayc_tweet_df)
bayc_tokens = big_string_bayc_tokens.split()
bayc_bigram_counts = Counter(ngrams(bayc_tokens, n=2))

# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

# Use token_count to get the top 10 words for Bitcoin
bayc_top_words = token_count(bayc_tokens,10)
print(bayc_top_words)

# Generate the Bitcoin word cloud
wc_bayc = WordCloud(width=1800, height=1000, max_words=50).generate(big_string_bayc_tokens)
plt.imshow(wc_bayc)